# Vertex AI function tests

NOTEBOOK TLDR:
1. The Google function calling by default uses functions as inputs. We can not send functions trought the api...
2. I found how this works and they turn the function into a **generativelanguage_v1beta.types.content.Tool.**
3. We can pass either the function or the **google.ai.generativelanguage_v1beta.types.content.Tool** as inputs for the tools list.
4. To implement this feature we need to find out when the client is using VertexAI (sdk side), and parse the functions into **google.ai.generativelanguage_v1beta.types.content.Tool** before sending it to the engine side.

Example: https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python#get-and-secure-api-key

Proto Format: https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python#generate-function-call

# 1. Raw example

### 1.1 Imports

In [6]:
import google.generativeai as genai

### 1.2 Define function
Here we define the function we are going to give the model. Don't forget to give typings to the the atributes, otherwise the function will fail.

In [2]:
# Google supported atributes. Dont run this cell
# AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])

In [7]:
def set_light_values(brightness: int, color_temp: int):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

### 1.3 Define model with tools
Here I want to show that the model works in both this cases:
1. Giving a function as a tool.
2. Giving a proto version of the function as a tool.

#### 1.3.1 Function as input

In [8]:
model_1 = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

In [9]:
model_1.generate_content('Dim the lights so the room feels cozy and warm.')

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "set_light_values",
                  "args": {
                    "brightness": 50.0,
                    "color_temp": "warm"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": 1,
          "index": 0,
          "safety_ratings": [
            {
              "category": 9,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 8,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 7,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 10,
       

#### 1.3.2 Proto function as input
A description of the function that allows it to be transmitable over an API.

In [11]:
proto_function = model_1._tools.to_proto()[0]
print(proto_function)

function_declarations {
  name: "set_light_values"
  description: "Set the brightness and color temperature of a room light. (mock API).\n\n    Args:\n        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness\n        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.\n\n    Returns:\n        A dictionary containing the set brightness and color temperature.\n    "
  parameters {
    type_: OBJECT
    properties {
      key: "color_temp"
      value {
        type_: INTEGER
      }
    }
    properties {
      key: "brightness"
      value {
        type_: INTEGER
      }
    }
    required: "brightness"
    required: "color_temp"
  }
}



In [12]:
model_2 = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[proto_function])

In [13]:
model_2.generate_content('Dim the lights so the room feels cozy and warm.')

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "set_light_values",
                  "args": {
                    "brightness": 50.0,
                    "color_temp": "warm"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": 1,
          "index": 0,
          "safety_ratings": [
            {
              "category": 9,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 7,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 10,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 8,
       

### 1.3.3 Calculator Example

In [14]:
calculator = {'function_declarations': [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]}

In [15]:
model_3 = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[calculator])

In [16]:
model_3.generate_content('What is 9 times 10?')

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "multiply",
                  "args": {
                    "a": 9.0,
                    "b": 10.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": 1,
          "index": 0,
          "safety_ratings": [
            {
              "category": 10,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 7,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 9,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 8,
              "probability": 1,
    

# 3. Conclusion

1. We need to understand when the user is sending a VertexAI request.
2. On the client side, we convert the function passed, to the proto format.
3. We make the call to the engine and get a response.

---

# LLMstudio Tests

In [19]:
calculator = {'function_declarations': [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]}

In [17]:
from llmstudio import LLM

In [18]:
llm = LLM('vertexai/gemini-1.5-flash')

In [20]:
llm.chat('What is 9 times 10?', functions = [calculator])

Functions in init.py sdk:  [{'function_declarations': [{'name': 'multiply', 'description': 'Returns the product of two numbers.', 'parameters': {'type_': 'OBJECT', 'properties': {'a': {'type_': 'NUMBER'}, 'b': {'type_': 'NUMBER'}}, 'required': ['a', 'b']}}]}]


Functions recieves in vertex.py skd: [{'function_declarations': [{'name': 'multiply', 'description': 'Returns the product of two numbers.', 'parameters': {'type_': 'OBJECT', 'properties': {'a': {'type_': 'NUMBER'}, 'b': {'type_': 'NUMBER'}}, 'required': ['a', 'b']}}]}]
Type of functions in vertex.py skd: <class 'list'>
Breakpoint: 1 - Model detects functions
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "multiply",
                  "args": {
                    "a": 9.0,
                    "b": 10.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": 1,
          "index": 0,
          "safety_ratings": [
            {
              "category": 10,
              "probability": 1,
      

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/uvicorn/protocols/http/h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/starlette/middleware/errors.py", line 1

HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:50001/api/engine/chat/vertexai

---

In [1]:
calculator = {'function_declarations': [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]}

In [2]:
print(type(calculator))

<class 'dict'>


In [3]:
from llmstudio import LLM

In [4]:
llm = LLM('vertexai/gemini-1.5-flash')

Running LLMstudio Tracking on http://localhost:50002 Running LLMstudio Engine on http://localhost:50001 

Functions recieves in vertex.py skd: [{'function_declarations': [{'name': 'multiply', 'description': 'Returns the product of two numbers.', 'parameters': {'type_': 'OBJECT', 'properties': {'a': {'type_': 'NUMBER'}, 'b': {'type_': 'NUMBER'}}, 'required': ['a', 'b']}}]}]
Type of functions in vertex.py skd: <class 'list'>
Breakpoint: 1 - Model detects functions
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "multiply",
                  "args": {
                    "a": 9.0,
                    "b": 10.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": 1,
          "index": 0,
      

In [5]:
llm.chat('What is 9 plus 10', functions = [calculator])

Functions in init.py sdk:  [{'function_declarations': [{'name': 'multiply', 'description': 'Returns the product of two numbers.', 'parameters': {'type_': 'OBJECT', 'properties': {'a': {'type_': 'NUMBER'}, 'b': {'type_': 'NUMBER'}}, 'required': ['a', 'b']}}]}]


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/uvicorn/protocols/http/h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/starlette/middleware/errors.py", line 1

HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:50001/api/engine/chat/vertexai

In [24]:
proto_function

function_declarations {
  name: "set_light_values"
  description: "Set the brightness and color temperature of a room light. (mock API).\n\n    Args:\n        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness\n        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.\n\n    Returns:\n        A dictionary containing the set brightness and color temperature.\n    "
  parameters {
    type_: OBJECT
    properties {
      key: "color_temp"
      value {
        type_: INTEGER
      }
    }
    properties {
      key: "brightness"
      value {
        type_: INTEGER
      }
    }
    required: "brightness"
    required: "color_temp"
  }
}

In [17]:
import json

proto_function_json = json.dumps(proto_function, default=str)
print(proto_function_json)

"function_declarations {\n  name: \"set_light_values\"\n  description: \"Set the brightness and color temperature of a room light. (mock API).\\n\\n    Args:\\n        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness\\n        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.\\n\\n    Returns:\\n        A dictionary containing the set brightness and color temperature.\\n    \"\n  parameters {\n    type_: OBJECT\n    properties {\n      key: \"color_temp\"\n      value {\n        type_: INTEGER\n      }\n    }\n    properties {\n      key: \"brightness\"\n      value {\n        type_: INTEGER\n      }\n    }\n    required: \"brightness\"\n    required: \"color_temp\"\n  }\n}\n"


In [16]:
from google.protobuf.json_format import MessageToJson
import json

# Assuming proto_function is your protobuf message
proto_function_json = MessageToJson(proto_function)
print(proto_function_json)

AttributeError: Unknown field for Tool: DESCRIPTOR

In [18]:
from google.protobuf.json_format import MessageToJson
import google.generativeai as genai

# Assuming proto_function is your protobuf message
proto_function = model_1._tools.to_proto()[0]
print(proto_function)

# Convert the protobuf message to JSON
try:
    proto_function_json = MessageToJson(proto_function)
    print(proto_function_json)
except AttributeError as e:
    print(f"Error: {e}")

function_declarations {
  name: "set_light_values"
  description: "Set the brightness and color temperature of a room light. (mock API).\n\n    Args:\n        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness\n        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.\n\n    Returns:\n        A dictionary containing the set brightness and color temperature.\n    "
  parameters {
    type_: OBJECT
    properties {
      key: "color_temp"
      value {
        type_: INTEGER
      }
    }
    properties {
      key: "brightness"
      value {
        type_: INTEGER
      }
    }
    required: "brightness"
    required: "color_temp"
  }
}

Error: Unknown field for Tool: DESCRIPTOR
